In [1]:
import cv2
import numpy as np

In [2]:
IMAGE_PATH = './img.jpg'

In [3]:
def viewImage(image, name_of_window="Image"):
    cv2.namedWindow(name_of_window, cv2.WINDOW_NORMAL)
    cv2.imshow(name_of_window, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## LABA1

In [15]:
image = cv2.imread(IMAGE_PATH)
viewImage(image, "Загруженное изображение")

In [16]:
image_g = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
viewImage(image_g, "Серое изображение")

In [17]:
ret, thresholds = cv2.threshold(image_g, 100, 255, cv2.THRESH_BINARY)
viewImage(thresholds)

In [18]:
result = cv2.bitwise_and(image, cv2.cvtColor(thresholds, cv2.COLOR_BAYER_GR2BGR))
viewImage(result)

## LABA2

http://robocraft.ru/page/opencv/